In [ ]:
"""
Cristian De Jesus Boone Posada

5° semestre

5BV1

ing. en inteligencia artificial

Este script en Python realiza un resumen extractivo automático de texto en base a múltiples métodos de procesamiento de lenguaje natural.
Cargar tres cartas desde archivos de texto, normaliza sus contenidos, y luego aplica seis métodos de resumen automático:
1. TF-IDF
2. Frecuencia de Palabras Normalizada
3. RAKE (Rapid Automatic Keyword Extraction)
4. TextRank utilizando spaCy
5. BERT (usando Transformers)
6. TextRank con Gensimm

Cada método extrae las frases más representativas de cada carta. Al final, los resúmenes generados para cada carta y método se imprimen en la consola.
Este código es útil para comparar y analizar diferentes técnicas de resumen automático de textos.
"""
#se dividio el codigo en 4 bloques para que la ejecucion sea mas rapida
# Importar librerías necesarias para el procesamiento de lenguaje natural y el resumen de textos
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from rake_nltk import Rake
from transformers import pipeline

# Descargar recursos de NLTK necesarios para tokenización, stopwords, y lematización
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Inicializar recursos de procesamiento de lenguaje
lemmatizer = WordNetLemmatizer()  # Lematizador para reducir palabras a su forma base
stop_words = set(stopwords.words('english'))  # Conjunto de palabras vacías en inglés
top_n_sentences = 3  # Número de frases representativas a incluir en cada resumen

# Función para normalizar texto (convertir a minúsculas, eliminar puntuación y palabras vacías, lematizar)
def normalizar_texto(texto):
    texto = texto.lower()  # Convertir a minúsculas
    texto = re.sub(r'[^\w\s]', '', texto)  # Eliminar puntuación
    palabras = nltk.word_tokenize(texto)  # Tokenizar el texto en palabras
    palabras_normalizadas = [lemmatizer.lemmatize(palabra) for palabra in palabras if palabra not in stop_words]
    return " ".join(palabras_normalizadas)  # Unir palabras normalizadas en un solo string

# Cargar y normalizar los documentos (cartas)
with open("C:\\Users\\crist\\Downloads\\Letter 1.txt", "r", encoding="utf-8") as file:
    carta1 = file.read()
carta1_normalizada = normalizar_texto(carta1)

with open("C:\\Users\\crist\\Downloads\\Letter 2.txt", "r", encoding="utf-8") as file:
    carta2 = file.read()
carta2_normalizada = normalizar_texto(carta2)

with open("C:\\Users\\crist\\Downloads\\Letter 3.txt", "r", encoding="utf-8") as file:
    carta3 = file.read()
carta3_normalizada = normalizar_texto(carta3)

# Función para resumen basado en TF-IDF
def resumen_tfidf(doc_text, top_n):
    sentences = nltk.sent_tokenize(doc_text)  # Dividir el documento en oraciones
    vectorizer = TfidfVectorizer()  # Crear un vectorizador TF-IDF
    X = vectorizer.fit_transform(sentences)  # Aplicar TF-IDF a las oraciones
    tfidf_sums = X.sum(axis=1).A1  # Sumar los valores TF-IDF de cada oración
    top_sentence_indices = tfidf_sums.argsort()[-top_n:][::-1]  # Seleccionar las oraciones con mayor puntaje
    return [sentences[i] for i in top_sentence_indices]  # Devolver las oraciones más importantes

# Función para resumen basado en frecuencia de palabras normalizada
def resumen_frecuencia(doc_text, top_n):
    sentences = nltk.sent_tokenize(doc_text)  # Dividir el documento en oraciones
    words = nltk.word_tokenize(doc_text.lower())  # Tokenizar palabras en minúsculas
    word_freq = Counter(words)  # Contar la frecuencia de cada palabra
    sentence_scores = {}  # Diccionario para almacenar el puntaje de cada oración
    for sentence in sentences:
        for word in nltk.word_tokenize(sentence.lower()):
            if word in word_freq:  # Sumar el puntaje de palabras en cada oración
                sentence_scores[sentence] = sentence_scores.get(sentence, 0) + word_freq[word]
    sorted_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)  # Ordenar por puntaje
    return sorted_sentences[:top_n]  # Devolver las oraciones con puntajes más altos

# Función para resumen usando RAKE (Rapid Automatic Keyword Extraction)
def resumen_rake(doc_text, top_n):
    r = Rake()  # Inicializar RAKE
    r.extract_keywords_from_text(doc_text)  # Extraer palabras clave del texto
    keywords = r.get_ranked_phrases()[:top_n]  # Obtener las frases más relevantes
    return keywords

# Función para resumen usando TextRank con Sumy
def resumen_textrank(doc_text, top_n):
    parser = PlaintextParser.from_string(doc_text, Tokenizer("english"))  # Preparar el texto para TextRank
    summarizer = TextRankSummarizer()  # Inicializar el sumario TextRank
    summary = summarizer(parser.document, top_n)  # Generar el resumen
    return [str(sentence) for sentence in summary]  # Devolver las oraciones del resumen

# Función para resumen usando BERT (Transformers) 
def resumen_bert(doc_text, top_n):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")  # Inicializar el sumario BERT
    summary = summarizer(doc_text, max_length=50 * top_n, min_length=25 * top_n, do_sample=False)
    return [summary[0]['summary_text']]  # Devolver el texto resumido

# Función para resumen usando LSA (Latent Semantic Analysis) con Sumy
def resumen_sumy_lsa(doc_text, top_n):
    parser = PlaintextParser.from_string(doc_text, Tokenizer("english"))  # Preparar el texto para LSA
    summarizer = LsaSummarizer()  # Inicializar el sumario LSA
    summary = summarizer(parser.document, top_n)  # Generar el resumen
    return [str(sentence) for sentence in summary]  # Devolver las oraciones del resumen



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\crist\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\crist\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\crist\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
# Generar resúmenes para Documento 1
print("Resúmenes para Documento 1:\n")

# Definir una lista de tuplas con cada función de resumen y su nombre correspondiente
for func, name in [
    (resumen_tfidf, "TF-IDF"),
    (resumen_frecuencia, "Frecuencia"),
    (resumen_rake, "RAKE"),
    (resumen_textrank, "TextRank"),
    (resumen_bert, "BERT"),
    (resumen_sumy_lsa, "LSA (Sumy)")
]:
    try:
        # Intentar generar el resumen usando la función actual y el documento normalizado (carta1_normalizada)
        print(f"\nResumen {name}:\n", " ".join(func(carta1_normalizada, top_n_sentences)))
    except Exception as e:
        # Si ocurre un error en el proceso de resumen, capturarlo y mostrar un mensaje
        print(f"Error en {name}:", e)

# Separador visual para diferenciar los resúmenes generados
print("\n" + "-"*50 + "\n")




Resúmenes para Documento 1:


Resumen TF-IDF:
 letter 1 _to mr saville england_ st petersburgh dec 11th 17 rejoice hear disaster accompanied commencement enterprise regarded evil foreboding arrived yesterday first task assure dear sister welfare increasing confidence success undertaking already far north london walk street petersburgh feel cold northern breeze play upon cheek brace nerve fill delight understand feeling breeze travelled region towards advancing give foretaste icy clime inspirited wind promise daydream become fervent vivid try vain persuaded pole seat frost desolation ever present imagination region beauty delight margaret sun ever visible broad disk skirting horizon diffusing perpetual splendour therefor leave sister put trust preceding navigatorsthere snow frost banished sailing calm sea may wafted land surpassing wonder beauty every region hitherto discovered habitable globe production feature may without example phenomenon heavenly body undoubtedly undiscovered solit

In [13]:
# Generar resúmenes para Documento 2
print("Resúmenes para Documento 2:\n")

# Definir una lista de tuplas con cada función de resumen y su nombre correspondiente
for func, name in [
    (resumen_tfidf, "TF-IDF"),
    (resumen_frecuencia, "Frecuencia"),
    (resumen_rake, "RAKE"),
    (resumen_textrank, "TextRank"),
    (resumen_bert, "BERT"),
    (resumen_sumy_lsa, "LSA (Sumy)")
]:
    try:
        # Intentar generar el resumen usando la función actual y el documento normalizado (carta2_normalizada)
        print(f"\nResumen {name}:\n", " ".join(func(carta2_normalizada, top_n_sentences)))
    except Exception as e:
        # Si ocurre un error en el proceso de resumen, capturarlo y mostrar un mensaje indicando el método
        print(f"Error en {name}:", e)

# Separador visual para diferenciar los resúmenes generados
print("\n" + "-"*50 + "\n")



Resúmenes para Documento 2:


Resumen TF-IDF:
 letter 2 _to mr saville england_ archangel 28th march 17 slowly time pass encompassed frost snow yet second step taken towards enterprise hired vessel occupied collecting sailor already engaged appear men depend certainly possessed dauntless courage one want never yet able satisfy absence object feel severe evil friend margaret glowing enthusiasm success none participate joy assailed disappointment one endeavour sustain dejection shall commit thought paper true poor medium communication feeling desire company man could sympathise whose eye would reply mine may deem romantic dear sister bitterly feel want friend one near gentle yet courageous possessed cultivated well capacious mind whose taste like approve amend plan would friend repair fault poor brother ardent execution impatient difficulty still greater evil selfeducated first fourteen year life ran wild common read nothing uncle thomas book voyage age became acquainted celebrated poet 

In [14]:
# Generar resúmenes para Documento 3
print("Resúmenes para Documento 3:\n")

# Definir una lista de tuplas con cada función de resumen y su nombre correspondiente
for func, name in [
    (resumen_tfidf, "TF-IDF"),
    (resumen_frecuencia, "Frecuencia"),
    (resumen_rake, "RAKE"),
    (resumen_textrank, "TextRank"),
    (resumen_bert, "BERT"),
    (resumen_sumy_lsa, "LSA (Sumy)")
]:
    try:
        # Intentar generar el resumen usando la función actual y el documento normalizado (carta3_normalizada)
        print(f"\nResumen {name}:\n", " ".join(func(carta3_normalizada, top_n_sentences)))
    except Exception as e:
        # Si ocurre un error en el proceso de resumen, capturarlo y mostrar un mensaje indicando el método
        print(f"Error en {name}:", e)

# Separador visual para diferenciar los resúmenes generados
print("\n" + "-"*50 + "\n")


Resúmenes para Documento 3:


Resumen TF-IDF:
 letter 3 _to mr saville england_ july 7th 17 dear sister write line haste say safeand well advanced voyage letter reach england merchantman homeward voyage archangel fortunate may see native land perhaps many year however good spirit men bold apparently firm purpose floating sheet ice continually pas u indicating danger region towards advancing appear dismay already reached high latitude height summer although warm england southern gale blow u speedily towards shore ardently desire attain breathe degree renovating warmth expected incident hitherto befallen u would make figure letter one two stiff gale springing leak accident experienced navigator scarcely remember record shall well content nothing worse happen u voyage adieu dear margaret assured sake well rashly encounter danger cool persevering prudent success _shall_ crown endeavour wherefore thus far gone tracing secure way pathless sea star witness testimony triumph still proceed unta